In [23]:
import time
import os
import pandas as pd
import datetime
import utilities
import numpy as np

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
LIST = 'daftar_anomali.xlsx'
SPV = 'wilayah_tugas_sak_82024.xlsx'

In [3]:
download_dir = os.getcwd()+r'\download'
chrome_options = Options()

prefs = {'download.default_directory': download_dir}
chrome_options.add_experimental_option('prefs', prefs)

In [4]:
driver = webdriver.Chrome(
    service=Service('chromedriver-win64/chromedriver.exe'),
    options=chrome_options
)

In [5]:
list_error = utilities.read_file(LIST)
list_error = list_error[list_error['link'].notna()]

In [6]:
driver.get('https://fasih-dashboard.bps.go.id/superset/sqllab')

# Manual Login

In [7]:
driver.get('https://fasih-dashboard.bps.go.id/superset/sqllab')
time.sleep(3)

selector = driver.find_element(
    By.XPATH,
    '//*[@id="a11y-query-editor-tabs"]/div[1]/div[1]/div'
)
tabs = selector.find_elements(
    By.XPATH,
    ".//button[@type='button' and @aria-label='remove']"
)

for i in range(1, len(tabs)+1):
    utilities.check_button(
        driver,
        f'//*[@id="a11y-query-editor-tabs"]/div[1]/div[1]/div/div[{1}]/button'
    )
    time.sleep(1)

In [8]:
anm = pd.DataFrame()
for index, row in list_error.iterrows():
    driver.get(row['link'])
    time.sleep(1)

    utilities.check_button(
        driver,
        '//*[@id="js-sql-toolbar"]/div[1]/span[1]/span/button'
    )
    time.sleep(1)

    while True:
        try:
            pending = driver.find_element(
                By.XPATH,
                ('/html/body/div[2]/div/div/div[2]/div/div/div/div[3]/div[3]'
                 '/div/div[2]/div/div[1]/div/span')
            ).text
            time.sleep(1)

            if pending != 'pending':
                break

        except:
            break
    time.sleep(1)

    while True:
        try:
            wait = driver.find_element(
                By.XPATH,
                ('/html/body/div[2]/div/div/div[2]/div/div/div/div[3]/div[3]'
                 '/div/div[2]/div/div[1]/div/div[3]/div/div/div[1]')
            ).text
            time.sleep(1)

            if wait != 'Running statement 1 out of 1':
                break

        except:
            break
    time.sleep(3)

    try:
        test = driver.find_element(
            By.XPATH,
            ('/html/body/div[2]/div/div/div[2]/div/div[1]/div/div[3]/div[3]'
             '/div/div[2]/div/div[1]/div/div[1]/div')
        ).text

        if test != 'The query returned no data':
            utilities.check_button(
                driver,
                '/html/body/div[2]/div/div/div[2]/div/div[1]/div/div[3]'
                '/div[3]/div/div[2]/div/div[1]/div/div[1]/div'
            )
            time.sleep(1)
            anm = pd.concat([
                anm,
                pd.DataFrame([{
                    'kode': row['kode'],
                    'keterangan': row['keterangan']
                }])
            ])

    except:
        pass

    utilities.check_button(
        driver,
        '//*[@id="a11y-query-editor-tabs"]/div[1]/div[1]/div/div[1]'
        '/button/span'
    )
    time.sleep(1)

In [12]:
file_df = pd.DataFrame({'download':os.listdir('download')})
file_df['date'] = (
    file_df['download']
    .str.split('_').str[-1]
    .str.split('.').str[0]
)
file_df['date'] = pd.to_datetime(file_df['date'], format='%Y%m%dT%H%M%S')
file_df = file_df.sort_values('date')

for i in range(len(file_df['download'])):
    filepath = f'download/{file_df.iloc[i,0]}'
    df=pd.read_csv(filepath)
    df['kode'] = anm.iloc[i,0]
    df['keterangan'] = anm.iloc[i,1]
    df = df[['kode', 'keterangan']+list(df.columns[:-2])]
    df.to_csv(filepath, index=False)

# Join with the person in charge

In [13]:
df = utilities.read_dir('download', 'Variable', 15, -1)
df['idbs'] = (
    df['KODE_PROV'] + df['KODE_KAB'].str.zfill(2) + df['KODE_KEC'].str.zfill(3)
    + df['KODE_DESA'].str.zfill(3) + df['BS'].str.zfill(4)
)

In [14]:
spv = utilities.read_file(SPV)
spv['pml'] = spv['pml'].str.split(',').str[0]

In [15]:
final = pd.merge(df, spv)

In [16]:
if not os.path.exists('anomali_petugas/'):
    os.makedirs('anomali_petugas/')

In [17]:
try:
    utilities.write_excel_width(final, 'pml', 'anomali_petugas')
except:
    utilities.write_excel(final, 'pml', 'anomali_petugas')

In [18]:
date = datetime.date.today().strftime("%Y-%m-%d")
final.to_excel(f'anomali_sak_{date}.xlsx', index=False, freeze_panes=(1, 1))

### Add last modified and status (Optional)

In [19]:
STS = 'sqllab_sak_2024_data_modification_20240902T011847.csv'
sts = utilities.read_file(STS, dtype=str)

In [63]:
final = pd.merge(pd.merge(df, spv), sts, on=['idbs', 'DSRT'])

#### Check catatan for Anomalies Code

In [64]:
final.loc[:, 'anomali'] = final['catatan'].str.upper().str.findall(
    r'ANOMALI_([A-Z]\d+)'
)

final.loc[:, 'anomali'] = final['anomali'].apply(
    lambda codes: ', '.join([
        f"{code[:-1]}{i+1}" for i, code in enumerate(codes)
    ]) if isinstance(codes, list) and codes else np.NaN
)

final.loc[:, 'kode_dalam_catatan'] = final.apply(
    lambda row: row['kode'] in row['anomali'].split(', ')
    if pd.notna(row['anomali']) else False, axis=1
)

In [65]:
final = final.loc[final['kode_dalam_catatan']==False]

In [68]:
try:
    utilities.write_excel_width(final, 'pml', 'anomali_petugas')
except:
    utilities.write_excel(final, 'pml', 'anomali_petugas')

In [70]:
date = datetime.date.today().strftime("%Y-%m-%d")
final.to_excel(f'anomali_sak_{date}.xlsx', index=False, freeze_panes=(1, 1))

### Create Summary for Supervisor

In [71]:
rekap = final.pivot_table(
    index=['pml', 'pcl'],
    columns='kode',
    aggfunc='size',
    fill_value=0
).reset_index()
rekap['total_anomali'] = rekap.iloc[:, 2:].sum(axis=1)
rekap = rekap[['pml', 'pcl', 'total_anomali']+list(rekap.columns[2:-1])]

rekap.to_excel(
    f'rekap_anomali_sak_{date}.xlsx',
    index=False,
    freeze_panes=(1, 2)
)